# SPLIT DEL DATASET Y FEATURE ENGINEERING

En este notebook vamos a tratar los diferentes aspectos explicados a continuación:

1. División del conjunto de datos en train y test

2. Combinación de variables: combinar variables que se consideren necesarias para guardar la misma información en una sola variable.

3. Estandarización: se vuelven a estandarizar los datasets puesto que se han realizado cambios.

4. Codificación de variables

5. Imputación de missings

### Importar librerías

En primer lugar, se importan las librerías necesarias y se cargan los datos guardados en los notebooks previos (tanto de train como test) para emplearlos en los modelos.

In [1]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import warnings
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

In [2]:
warnings.filterwarnings('ignore')

Cargamos los datasets guardados en el notebook 2.
Sin embargo, vamos a utilizar en primer lugar el dataset 'pd_fraud_reduced'. Si a continuación consideramos necesario utilizar el dataset 'pd_fraud_selection', lo usaremos.

In [3]:
pd_fraud_reduced = pd.read_csv("data/pd_fraud_reduced.csv")
pd_fraud_selection = pd.read_csv("data/pd_fraud_selection.csv")

Eliminamos la variable "Unnamed: 0" de ambos dataframes que se ha creado automáticamente al guardar los datasets en el notebook anterior.

In [4]:
pd_fraud_reduced =pd_fraud_reduced.drop(columns=["Unnamed: 0"])
pd_fraud_selection =pd_fraud_selection.drop(columns=["Unnamed: 0"])

## División del dataset en train, validation y test

En primer lugar, antes de realizar el feature engineering procedemos a la división del conjunto de datos de ambos datasets.

En el contexto del aprendizaje automático, los conjuntos de entrenamiento (train), validación (validation) y prueba (test) se utilizan para entrenar, ajustar y evaluar modelos predictivos.

- Conjunto de Entrenamiento: Este conjunto de datos se utiliza para entrenar el modelo. El algoritmo de aprendizaje utiliza patrones en este conjunto para ajustar los parámetros del modelo y aprender las relaciones entre las características y las etiquetas.

- Conjunto de Validación: Después de entrenar el modelo en el conjunto de entrenamiento, se utiliza el conjunto de validación para ajustar los hiperparámetros del modelo y seleccionar el mejor modelo entre varios candidatos. El conjunto de validación proporciona una evaluación independiente del rendimiento del modelo y ayuda a evitar el sobreajuste (overfitting).

- Conjunto de Prueba: Este conjunto se utiliza para evaluar el rendimiento final del modelo después de que ha sido entrenado y ajustado. El conjunto de prueba simula datos "nuevos" que el modelo no ha visto durante el entrenamiento ni durante la validación. Proporciona una estimación no sesgada del rendimiento del modelo en datos no vistos y ayuda a evaluar la capacidad del modelo para generalizar a nuevas instancias.

[Información adicional](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

Split de los datasets 'pd_fraud_reduced' y 'pd_fraud_selection'.

En este notebook **únicamente realizaremos la división en train y test** para los dos conjuntos de datos, puesto que la obtención del conjunto de validación nos parece más correcta incluirla previamente al muestreo de los conjuntos de datos (estratificación, undersampling y oversampling). De esta forma, en la siguientes líneas de código se realiza el split otorgando una porporción del 20% al test y un 80% al train (tanto para 'pd_fraud_reduced' como para 'pd_fraud_selection'):

In [5]:
X_pd_fraud_red_train, X_pd_fraud_red_test, y_pd_fraud_red_train, y_pd_fraud_red_test = train_test_split(pd_fraud_reduced.drop('fraud_bool',axis=1), 
                                                                     pd_fraud_reduced['fraud_bool'], 
                                                                     stratify=pd_fraud_reduced['fraud_bool'], 
                                                                     test_size=0.2)

Ahora hacemos lo mismo con el conjunto de datos 'pd_fraud_selection'.

In [6]:
X_pd_fraud_sel_train, X_pd_fraud_sel_test, y_pd_fraud_sel_train, y_pd_fraud_sel_test = train_test_split(pd_fraud_selection.drop('fraud_bool',axis=1), 
                                                                     pd_fraud_selection['fraud_bool'], 
                                                                     stratify=pd_fraud_selection['fraud_bool'], 
                                                                     test_size=0.2)

## Feature Engineering: Codificación de las variables categóricas, escalado y modelo

## Combinación de variables

Dentro del apartado de combinación de variables y tal y como hemos comentado anteriormente, consideramos adecuado crear una nueva variable llamada 'mean_velocities' que recoja las tres velocidades en el dataset 'pd_fraud_reduced': 'velocity_6h', 'velocity_24h', y 'velocity_4w'. Para ello, la nueva variable empleada será la media aritmética de las tres velocidades puesto que consideramos que este tipo de operación recoge la información de la forma más fiable posible.

In [7]:
mean_velocities = (X_pd_fraud_red_train['velocity_6h'] + X_pd_fraud_red_train['velocity_24h'] + X_pd_fraud_red_train['velocity_4w']) / 3

X_pd_fraud_red_train['mean_velocities'] = mean_velocities

X_pd_fraud_red_train                                                       

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,velocity_6h,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,mean_velocities
658399,0.1,0.137525,-1.0,159.0,50,0.019500,-1.539034,2,1449.0,7769.672778,...,0,500.0,0,0,8.413229,1,0,1,0,6732.736893
827702,0.8,0.544001,-1.0,69.0,20,0.019971,-0.729813,3,717.0,2326.470239,...,0,200.0,0,0,6.699359,4,0,1,5,2989.737039
904694,0.9,0.691325,113.0,11.0,30,0.006098,-0.425804,2,420.0,5699.936869,...,1,2000.0,0,0,5.424367,1,0,1,4,5219.869368
62708,0.6,0.476041,-1.0,42.0,40,0.035810,32.541154,0,1145.0,1688.332873,...,0,200.0,0,0,10.212300,0,1,1,7,2772.296037
828641,0.8,0.896029,216.0,11.0,50,0.015699,-1.212647,3,2904.0,5181.983067,...,0,1000.0,0,0,24.210540,1,1,1,5,4475.233434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895651,0.1,0.539465,-1.0,295.0,20,0.037646,-1.269443,1,1311.0,4700.507384,...,0,200.0,0,0,4.292226,4,0,1,4,4867.661102
301979,0.7,0.311811,31.0,13.0,30,0.004643,15.030462,1,2914.0,10160.171219,...,0,200.0,0,0,3.458866,1,0,1,2,6707.938740
508234,0.5,0.328098,-1.0,231.0,50,0.022487,-1.362494,2,2313.0,12774.310725,...,1,200.0,0,0,27.693601,0,1,1,1,8999.014907
993930,0.7,0.823786,-1.0,121.0,20,0.034040,-1.151798,1,1281.0,4631.573025,...,1,200.0,0,0,2.986301,4,0,1,4,4510.806503


Al haber creado en el dataset 'pd_fraud_reduced' esta combinación de variables consideramos redundante que permanezcan las variables originales, y por tanto se eliminan 'velocity_6h', 'velocity_24h', 'velocity_4w' de este conjunto de datos y así evitar la duplicidad de la información:

In [8]:
X_pd_fraud_red_train = X_pd_fraud_red_train.drop(['velocity_6h', 'velocity_24h', 'velocity_4w'], axis=1)
X_pd_fraud_red_train

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,bank_branch_count_8w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,mean_velocities
658399,0.1,0.137525,-1.0,159.0,50,0.019500,-1.539034,2,1449.0,0.0,...,0,500.0,0,0,8.413229,1,0,1,0,6732.736893
827702,0.8,0.544001,-1.0,69.0,20,0.019971,-0.729813,3,717.0,22.0,...,0,200.0,0,0,6.699359,4,0,1,5,2989.737039
904694,0.9,0.691325,113.0,11.0,30,0.006098,-0.425804,2,420.0,0.0,...,1,2000.0,0,0,5.424367,1,0,1,4,5219.869368
62708,0.6,0.476041,-1.0,42.0,40,0.035810,32.541154,0,1145.0,562.0,...,0,200.0,0,0,10.212300,0,1,1,7,2772.296037
828641,0.8,0.896029,216.0,11.0,50,0.015699,-1.212647,3,2904.0,8.0,...,0,1000.0,0,0,24.210540,1,1,1,5,4475.233434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895651,0.1,0.539465,-1.0,295.0,20,0.037646,-1.269443,1,1311.0,1.0,...,0,200.0,0,0,4.292226,4,0,1,4,4867.661102
301979,0.7,0.311811,31.0,13.0,30,0.004643,15.030462,1,2914.0,540.0,...,0,200.0,0,0,3.458866,1,0,1,2,6707.938740
508234,0.5,0.328098,-1.0,231.0,50,0.022487,-1.362494,2,2313.0,1.0,...,1,200.0,0,0,27.693601,0,1,1,1,8999.014907
993930,0.7,0.823786,-1.0,121.0,20,0.034040,-1.151798,1,1281.0,1899.0,...,1,200.0,0,0,2.986301,4,0,1,4,4510.806503


Con todas estas modificaciones ya tendríamos el dataset previo a la fase de feature engineering tras haber realizado una selección de variables inicial en base al análisis exploratorio de los datos y las regularizaciones Ridge y Lasso.

## Estandarización

Volvemos a realizar la estandarización sobre los dos notebooks puesto que es el 'pd_fraud_reduced' hemos realizado cambios, y en el anterior notebook no habíamos estandarizado las variables que finalmente incluiremos en 'pd_fraud_reduced' y 'pd_fraud_selection':

En primer lugar estandarizamos el conjunto de datos de train de 'pd_fraud_reduced' ('X_pd_fraud_red_train') y con esos datos estandarizados creamos un dataframe llamado 'X_pd_fraud_red_train_std':

In [9]:
escala = StandardScaler()
a = escala.fit_transform(X_pd_fraud_red_train)
X_pd_fraud_red_train_std = pd.DataFrame(a, columns= X_pd_fraud_red_train.columns)

In [10]:
X_pd_fraud_red_train_std.describe().T

,count,mean,std,min,25%,50%,75%,max
income,800000.0,-6.783907e-17,1.000001,-1.593209,-0.904427,0.128746,0.817528,1.161919
name_email_similarity,800000.0,4.979483e-16,1.000001,-1.706707,-0.928083,-0.005856,0.906079,1.751790
prev_address_months_count,800000.0,2.298162e-17,1.000001,-0.402198,-0.402198,-0.402198,-0.106941,8.319232
current_address_months_count,800000.0,1.450395e-17,1.000001,-0.991132,-0.765009,-0.391907,0.489970,3.859195
customer_age,800000.0,-2.884804e-16,1.000001,-1.970996,-1.139056,-0.307116,0.524824,4.684524
days_since_request,800000.0,3.093970e-17,1.000001,-0.190339,-0.188999,-0.187513,-0.185436,14.016082
intended_balcon_amount,800000.0,-4.439116e-17,1.000001,-1.195120,-0.486642,-0.469314,-0.177685,5.125510
payment_type,800000.0,7.721823e-17,1.000001,-1.274627,-1.274627,-0.240028,0.794571,2.863769
zip_count_4w,800000.0,-2.556177e-17,1.000001,-1.564012,-0.674621,-0.308519,0.369964,5.100446
bank_branch_count_8w,800000.0,-1.103118e-17,1.000001,-0.401681,-0.399507,-0.382112,-0.347323,4.784082


Se comprueba que las variables del nuevo data frame estandarizado 'X_pd_fraud_red_train_std' cuentan con media 0 y desviación típica 1

Realizamos el mismo procedimiento de estandarización con el conjunto de datos de train de 'pd_fraud_selection' ('X_pd_fraud_sel_train') recogiendo dichos datos estandarizados en el dataframe 'X_pd_fraud_sel_train_std':

In [11]:
escala = StandardScaler()
a = escala.fit_transform(X_pd_fraud_sel_train)
X_pd_fraud_sel_train_std = pd.DataFrame(a, columns= X_pd_fraud_sel_train.columns)

In [12]:
X_pd_fraud_sel_train_std.describe().T

,count,mean,std,min,25%,50%,75%,max
income,800000.0,9.933387e-17,1.000001,-1.592303,-0.903724,0.129144,0.817723,1.162013
name_email_similarity,800000.0,4.936584e-16,1.000001,-1.708180,-0.927718,-0.005448,0.904980,1.752266
prev_address_months_count,800000.0,3.218759e-17,1.000001,-0.402185,-0.402185,-0.402185,-0.107129,8.313325
customer_age,800000.0,-4.689582e-17,1.000001,-1.969250,-1.138063,-0.306876,0.524311,4.680245
employment_status,800000.0,-5.408118e-17,1.000001,-0.474390,-0.474390,-0.474390,0.305847,4.207029
credit_risk_score,800000.0,-6.744827e-17,1.000001,-4.319513,-0.688696,-0.129004,0.674655,3.702728
email_is_free,800000.0,2.799538e-17,1.000001,-1.061884,-1.061884,0.941723,0.941723,0.941723
housing_status,800000.0,-1.102496e-16,1.000001,-1.395554,-0.607123,0.181309,0.181309,3.335035
phone_home_valid,800000.0,-5.130119e-17,1.000001,-0.845684,-0.845684,-0.845684,1.182475,1.182475
has_other_cards,800000.0,-1.006928e-16,1.000001,-0.535391,-0.535391,-0.535391,-0.535391,1.867795


Se comprueba que las variables del nuevo data frame estandarizado 'X_pd_fraud_sel_train_std' cuentan con media 0 y desviación típica 1

Al haber estandarizado los datos se facilita la interpretación de los datos, se evita la dominancia de características y además, en algunos algoritmos sensibles a la escala como Support Vector Machine (SVM) se ayuda a que funcionen mejor.

## Imputación de missings

Como vimos en anteriores notebooks, hay algunas variables que presentan missings con valor -1. Sin embargo, la variable 'intended_balcon_amount' recoge sus valores faltantes como todos los valores negativos que presente, no siempre el mismo valor (como es el caso del resto de variables que presentan valores faltantes). 

Únicamente se van a imputar los missings de la variable cuyos missings son todos los valores negativos puesto que consideramos que las variables con missings = -1 ya están codificadas e imputarlos por otro valor (por ejemplo la mediana) haría que nuestros datos estuviesen sesgados.

Destacar que únicamente se van a imputar los missings en los dataframes 'pd_fraud_reduced' y su escalado 'X_pd_fraud_red_train_std' puesto que en los dataframe 'pd_fraud_selection' y 'X_pd_fraud_sel_train_std' esta variable no ha sido escogida.

In [13]:
def imputar_negativos_por_menos_uno(df, columnas_a_imputar):
    for columna in columnas_a_imputar:
        if columna in df.columns:
            df[columna] = np.where(df[columna] < 0, -1, df[columna])
    return df

columnas_a_imputar_negativos = ['intended_balcon_amount']

Esta función toma un DataFrame 'df' y una lista de nombres de columnas 'columnas_a_imputar' iterando sobre estas columnas y utiliza np.where para reemplazar los valores negativos por -1 en esas columnas específicas; en concreto en la columna 'intended_balcon_amount'.

A continuación, imputamos los missings tanto en 'X_pd_fraud_red_train_std' y 'X_pd_fraud_red_train':

In [14]:
X_pd_fraud_red_train_std = imputar_negativos_por_menos_uno(X_pd_fraud_red_train_std, columnas_a_imputar_negativos)
X_pd_fraud_red_train = imputar_negativos_por_menos_uno(X_pd_fraud_red_train, columnas_a_imputar_negativos)

A continuación se muestran los dos nuevos datasets tras la imputación de los missings en la variable 'intended_balcon_amount':

In [ ]:
X_pd_fraud_red_train_std

In [15]:
X_pd_fraud_red_train

,income,name_email_similarity,prev_address_months_count,current_address_months_count,customer_age,days_since_request,intended_balcon_amount,payment_type,zip_count_4w,bank_branch_count_8w,...,has_other_cards,proposed_credit_limit,foreign_request,source,session_length_in_minutes,device_os,keep_alive_session,device_distinct_emails_8w,month,mean_velocities
658399,0.1,0.137525,-1.0,159.0,50,0.019500,-1.000000,2,1449.0,0.0,...,0,500.0,0,0,8.413229,1,0,1,0,6732.736893
827702,0.8,0.544001,-1.0,69.0,20,0.019971,-1.000000,3,717.0,22.0,...,0,200.0,0,0,6.699359,4,0,1,5,2989.737039
904694,0.9,0.691325,113.0,11.0,30,0.006098,-1.000000,2,420.0,0.0,...,1,2000.0,0,0,5.424367,1,0,1,4,5219.869368
62708,0.6,0.476041,-1.0,42.0,40,0.035810,32.541154,0,1145.0,562.0,...,0,200.0,0,0,10.212300,0,1,1,7,2772.296037
828641,0.8,0.896029,216.0,11.0,50,0.015699,-1.000000,3,2904.0,8.0,...,0,1000.0,0,0,24.210540,1,1,1,5,4475.233434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895651,0.1,0.539465,-1.0,295.0,20,0.037646,-1.000000,1,1311.0,1.0,...,0,200.0,0,0,4.292226,4,0,1,4,4867.661102
301979,0.7,0.311811,31.0,13.0,30,0.004643,15.030462,1,2914.0,540.0,...,0,200.0,0,0,3.458866,1,0,1,2,6707.938740
508234,0.5,0.328098,-1.0,231.0,50,0.022487,-1.000000,2,2313.0,1.0,...,1,200.0,0,0,27.693601,0,1,1,1,8999.014907
993930,0.7,0.823786,-1.0,121.0,20,0.034040,-1.000000,1,1281.0,1899.0,...,1,200.0,0,0,2.986301,4,0,1,4,4510.806503


## Codificación del resto de variables categóricas

Puesto que para la realización de las regularizaciones de Ridge y Lasso era necesario pasar las variables categóricas a numéricas, ya hicimos una transformación de las mismas a mano. Por lo tanto, en el presente notebook ya no va a ser necesario realizar técnicas como el OneHotEncoding.

Por tanto, guardamos todos los datasets creados y modificados en este notebook para posteriormente poder emplearlos en futuros notebooks como en el de los modelos.

In [16]:
X_pd_fraud_red_train.to_csv("data/X_pd_fraud_red_train.csv")
X_pd_fraud_red_train_std.to_csv("data/X_pd_fraud_red_train_std.csv")
X_pd_fraud_red_test.to_csv("data/X_pd_fraud_red_test.csv")
y_pd_fraud_red_train.to_csv("data/y_pd_fraud_red_train.csv")
y_pd_fraud_red_test.to_csv("data/y_pd_fraud_red_test.csv")

X_pd_fraud_sel_train.to_csv("data/X_pd_fraud_sel_train.csv")
X_pd_fraud_sel_train_std.to_csv("data/X_pd_fraud_sel_train_std.csv")
X_pd_fraud_sel_test.to_csv("data/X_pd_fraud_sel_test.csv")
y_pd_fraud_sel_train.to_csv("data/y_pd_fraud_sel_train.csv")
y_pd_fraud_sel_test.to_csv("data/y_pd_fraud_sel_test.csv")

# CONCLUSIONES FINALES

- En primer lugar, se ha realizado la **división de los datasets 'pd_fraud_reduced' y 'pd_fraud_selection' en conjunto de entrenamiento y test (80% y 20%, respectivamente)**. La división del conjunto de entrenamiento en train y validación se efectuará previamente al muestreo, en el siguiente notebook.

- Se ha realizado como **combinación de variables** en el dataset 'pd_fraud_reduced' la media aritmética de 'velocity_6h', 'velocity_24h', y 'velocity_4w' en una nueva variable llamada 'mean velocities'. Por tanto, las variables originales han sido eliminadas del conjunto de datos para evitar la duplicidad. 

- Se ha **estandarizado el conjunto de entrenamiento de los datasets 'pd_fraud_reduced' y 'pd_fraud_selection'** almacenándose dichos datos estandarizados en 'X_pd_fraud_red_train_std' y 'X_pd_fraud_sel_train_std', respectivamente. 

- Respecto a la **imputación de missings** se van a imputar únicamente los correspondientes a la variable 'intended_balcon_amount' (todos sus valores negativos) dado que el resto de variables con valores ausentes ya están codificadas e imputarlos por otro valor como puede ser la  mediana haría que nuestros datos estuviesen sesgados. Dicha imputación se realizará en los dataframes que disponen de esta variable, 'pd_fraud_reduced' y su escalado 'X_pd_fraud_red_train_std'.

- Dado que para la regularizaciones de  Ridge y Lasso fue necesaria la **codificación de las variables categóricas** en este notebook no ha sido necesario realizarlo de nuevo.